# AIDM7380 Recommender Systems for Digital Media


*   Student Name: ZHANG Huiyi
*   Student Number: 24474940



In [2]:
# Install libraries using pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install numpy

Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/pip/__main__.py", line 22, in <module>
    from pip._internal.cli.main import main as _main
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/main.py", line 11, in <module>
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/main_parser.py", line 9, in <module>
    from pip._internal.build_env import get_runnable_pip
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/build_env.py", line 15, in <module>
    from pip._vendor.packaging.requirements import Requirement
  File "/usr/local/lib/python3.11/dist-packages/pip/_vendor/packaging/requirements.py", line 8, in <module>
    from ._parser impo

# User Behaviour and the User-Item Matrix

## Importing and knowing your data

In [4]:
import pandas as pd
import numpy as np

In [5]:
from google.colab import drive
drivePath = '/content/drive' #please do not change
drive.mount(drivePath)

Mounted at /content/drive


In [6]:
# Install the library on your environment
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=b6bf834b8a5e0f1b7112cba492075732268c91b116bc756c409af9aa52bd5a7d
  Stored in directory: /root/.cache/pip/wheels/40/b3/0f/a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget


In [7]:
# Import the library
import wget
import os, pathlib

# Setup URL and path variables
baseURL = 'https://raw.githubusercontent.com/pmengoni/AIDM7380-2223S2/main/'
doc = 'collector_log.csv'
fullURL = baseURL + doc

dataPath = drivePath + '/MyDrive/Colab Notebooks/data'

#Create path if not exists
if not(os.path.exists(dataPath)):
  path = pathlib.Path(dataPath)
  path.mkdir(parents=True, exist_ok=True)
else:
  print('The data path you selected already exists')

# Download the file
fileName = wget.download(fullURL, out=dataPath)

# Print the file name including the local path
print(fileName)

The data path you selected already exists
/content/drive/MyDrive/Colab Notebooks/data/collector_log (1).csv


In [8]:
evidence = pd.read_csv(fileName)

In [9]:
# checkin the type and take a glance at the head
print(type(evidence))
evidence.head(5)

<class 'pandas.core.frame.DataFrame'>


,id,created,content_id,event,session_id,user_id
0,3,14/01/2020 17:54,4501244,details,794773,400003
1,4,14/01/2020 17:54,3521164,moreDetails,794773,400003
2,5,14/01/2020 17:54,3640424,details,441002,400005
3,6,14/01/2020 17:54,2823054,moreDetails,885440,400001
4,7,14/01/2020 17:54,3553976,genreView,441003,400005


## Examining the attributes of the Data Frame (standard procedures)

- ```df.shape``` ("dim" in R)
- ```df.columns``` (check the variables, like "names" in R)
- ```df.index``` (check the index of the "rows")
- ```df.info()```
- ```df.describe()``` (descriptive statistics for numerical variables)

In [21]:
evidence.shape
# (the number of cases/observations, the number of variables)

(100000, 6)

In [22]:
evidence.columns

Index(['id', 'created', 'content_id', 'event', 'session_id', 'user_id'], dtype='object')

In [23]:
evidence.index

RangeIndex(start=0, stop=100000, step=1)

In [24]:
evidence.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   id          100000 non-null  int64 
 1   created     100000 non-null  object
 2   content_id  100000 non-null  int64 
 3   event       100000 non-null  object
 4   session_id  100000 non-null  int64 
 5   user_id     100000 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 4.6+ MB


In [25]:
evidence.describe()

,id,content_id,session_id,user_id
count,100000.000000,1.000000e+05,100000.000000,100000.000000
mean,50002.500000,2.987698e+06,585135.625220,400003.502640
std,28867.657797,1.230240e+06,317096.306234,1.703597
min,3.000000,4.752900e+05,42450.000000,400001.000000
25%,25002.750000,1.972591e+06,404784.000000,400002.000000
50%,50002.500000,3.040964e+06,794789.000000,400004.000000
75%,75002.250000,4.034228e+06,886267.000000,400005.000000
max,100002.000000,5.700672e+06,935091.000000,400006.000000


In [26]:
users = evidence.user_id.unique()
content = evidence.content_id.unique()
print(type(content))
print(len(content))
print(len(users))

<class 'numpy.ndarray'>
103
6


# Implicit Ratings
## Binary Matrix
Let's create a user-item binary matrix from the "buy" events

In [27]:
#Create a user-item binary matrix
uiBuyMatrix = pd.DataFrame(columns=content, index=users)
uiBuyMatrix.head(2)

,4501244,3521164,3640424,2823054,3553976,3470600,4513674,4698684,3315342,3874544,...,4196776,2948356,1355644,3300542,5247022,2140479,1083452,1179933,3410834,3553442
400003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
400005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
evidence.event.unique()

array(['details', 'moreDetails', 'genreView', 'addToList', 'buy'],
      dtype=object)

Select only the "buy" events

In [29]:
buyEvidence = evidence[evidence['event'] == 'buy']
buyEvidence.head(5)

,id,created,content_id,event,session_id,user_id
92,95,14/01/2020 17:54,4501244,buy,794776,400003
131,134,14/01/2020 17:54,2937696,buy,885441,400001
358,361,14/01/2020 17:54,3874544,buy,885444,400001
612,615,14/01/2020 17:54,3949660,buy,885445,400001
707,710,14/01/2020 17:54,5512872,buy,42460,400006


Create the user-item matrix `uiBuyMatrix` for the buy events

In [30]:
for index, row in buyEvidence.iterrows():
    currentUser = row['user_id']
    currentContent = row['content_id']
    uiBuyMatrix.at[currentUser, currentContent] = 1

In [ ]:
print(uiBuyMatrix)

       4501244 3521164 3640424 2823054 3553976 3470600 4513674 4698684  \
400003       1       1       1       1       1       1       1       1   
400005     NaN     NaN       1     NaN       1     NaN       1     NaN   
400001       1       1       1     NaN       1       1       1       1   
400006       1     NaN       1       1     NaN       1       1       1   
400002     NaN     NaN       1       1       1       1       1     NaN   
400004     NaN     NaN       1     NaN     NaN     NaN     NaN     NaN   

       3315342 3874544  ... 4196776 2948356 1355644 3300542 5247022 2140479  \
400003       1       1  ...     NaN     NaN       1     NaN     NaN       1   
400005       1     NaN  ...     NaN     NaN       1     NaN     NaN       1   
400001       1       1  ...     NaN     NaN       1     NaN       1       1   
400006     NaN       1  ...     NaN     NaN     NaN     NaN       1     NaN   
400002       1     NaN  ...       1       1       1     NaN       1     NaN   
400004 

## Behavioural Implicit Ratings

Using the formula introduced during lecture

$${IR}_(i,u) = \left(w_1*{\#event}_1\right)+\left(w_2*{\#event}_2\right)+\dots+\left(w_n*{\#event}_n\right)$$

In [31]:
#Create a user-item matrix
uiMatrix = pd.DataFrame(columns=content, index=users)
uiMatrix.head(2)

,4501244,3521164,3640424,2823054,3553976,3470600,4513674,4698684,3315342,3874544,...,4196776,2948356,1355644,3300542,5247022,2140479,1083452,1179933,3410834,3553442
400003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
400005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Type of events recorded in the logs

In [32]:
eventTypes = evidence.event.unique()
print(eventTypes)

['details' 'moreDetails' 'genreView' 'addToList' 'buy']


Give a weight to each of them

In [33]:
eventWeights = {
    'details': 15,
    'moreDetails': 50,
    'genreView': 0,
    'addToList': 0,
    'buy': 100}

Compute the Implicit Rating for each user-item combination.
Populate the user-item matrix `uiMatrix` with the IR values.

In [34]:
# Iterate the evidence
for index, row in evidence.iterrows():
    # Select the user and items involved
    currentUser = row['user_id']
    currentContent = row['content_id']

    # Extract the appropriate weight for the event
    w = eventWeights[row['event']]

    # Find the value eventually stored for the current user-item combination
    currentValue = uiMatrix.at[currentUser, currentContent]
    if np.isnan(currentValue):
        currentValue = 0

    # Compute the new value and update the user-item matrix
    updatedValue = currentValue + w #+ (1 * w)
    uiMatrix.at[currentUser, currentContent] = updatedValue


In [35]:
print(uiMatrix)

       4501244 3521164 3640424 2823054 3553976 3470600 4513674 4698684  \
400003    4820    4645    4110    3875    2785    4165    7070    4155   
400005     NaN     NaN    7555     NaN    7785     NaN    7840     NaN   
400001    4810    4590    4255    4000    2335    5230    6825    4520   
400006    7615    7675     195    8700     NaN    9240    8835    8600   
400002    2245    2420    5890    2815    1710    2345    4015    1900   
400004     NaN     NaN    8320     NaN     NaN     NaN     NaN     NaN   

       3315342 3874544  ... 4196776 2948356 1355644 3300542 5247022 2140479  \
400003    3920    4200  ...    1625    2040    2745    1620    3580    2840   
400005    7965     NaN  ...      50      80    7630      15     NaN    8090   
400001    4235    4445  ...    1630    1715    2150    1570    4600    2445   
400006      30    9540  ...      60     230     NaN     150    7725     NaN   
400002    6160    2160  ...    4800    4060    1665    3630    2530    2090   
400004 

# Exercise 1 (1')
Update the user-item matrix by normalizing the values between 0 and 10. **Note**: NaN values should be maintained as NaN

**Hint**: the maximum value in the matrix is the following value

In [36]:
nanmaxValue = np.nanmax(uiMatrix.values)

In [37]:
uiMatrix_normalized = uiMatrix.applymap(lambda x: (x / nanmaxValue) * 10 if not np.isnan(x) else np.nan)

print(uiMatrix_normalized)

         4501244   3521164   3640424   2823054   3553976   3470600   4513674  \
400003  4.951207  4.771443  4.221880  3.980483  2.860812  4.278377  7.262455   
400005       NaN       NaN  7.760657       NaN  7.996918       NaN  8.053416   
400001  4.940935  4.714946  4.370827  4.108885  2.398562  5.372368  7.010786   
400006  7.822291  7.883924  0.200308  8.936826       NaN  9.491525  9.075501   
400002  2.306112  2.485876  6.050334  2.891628  1.756549  2.408834  4.124294   
400004       NaN       NaN  8.546482       NaN       NaN       NaN       NaN   

         4698684   3315342   3874544  ...   4196776   2948356   1355644  \
400003  4.268105  4.026708  4.314330  ...  1.669235  2.095532  2.819723   
400005       NaN  8.181818       NaN  ...  0.051361  0.082178  7.837699   
400001  4.643041  4.350282  4.565999  ...  1.674371  1.761685  2.208526   
400006  8.834104  0.030817  9.799692  ...  0.061633  0.236261       NaN   
400002  1.951721  6.327684  2.218798  ...  4.930663  4.170519  1

<ipython-input-37-e1107c96e186>:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  uiMatrix_normalized = uiMatrix.applymap(lambda x: (x / nanmaxValue) * 10 if not np.isnan(x) else np.nan)


# Exercise 2 (1')
Limit the number of relevant events to a specific threshold (e.g. 10).

In [38]:
newMatrix = pd.DataFrame(columns=content, index=users)
limitNumber = 10

for user in users:
  for item in content:
    userEvidence = evidence['user_id'] == user
    contentEvidence = evidence['content_id'] == item
    userItemEvents = evidence[userEvidence & contentEvidence]

    if len(userItemEvents) > limitNumber:
      userItemEvents = userItemEvents.head(limitNumber)

    updatedValue = 0
    for index, row in userItemEvents.iterrows():
        w = eventWeights[row['event']]
        updatedValue += w

    newMatrix.at[user, item] = updatedValue

print(newMatrix)


       4501244 3521164 3640424 2823054 3553976 3470600 4513674 4698684  \
400003     310     290     225     210     140     230     240     275   
400005       0       0     125       0     240       0     310       0   
400001     145     190     190     155     215     175     125     130   
400006     160      90     195     175       0     310     175     295   
400002     280     205     175     260     310     260     120     190   
400004       0       0     240       0       0       0       0       0   

       3315342 3874544  ... 4196776 2948356 1355644 3300542 5247022 2140479  \
400003     125     225  ...     225     210     325     175     210     125   
400005     125       0  ...      50      80     105      15       0     275   
400001     225     245  ...     260     175     125     225     225     155   
400006      30     140  ...      60     230       0     150     295       0   
400002     225     190  ...     240     325     215     140     230     295   
400004 

# Exercise 3 (2')
Add a decay threshold. Older events are not informative about the user's behavior.
Check the sample Python function and adapt the code according to the following formulation.

## Behavioural Implicit Ratings with Decay

We modify the formula introduced during lecture

$${IR}_{(i,u)} = \sum_{i=1}^n w_i*{\#event}_i = \left(w_1*{\#event}_1\right)+\left(w_2*{\#event}_2\right)+\dots+\left(w_n*{\#event}_n\right)$$

to

$${IRDecay}_{(i,u)} = \sum_{i=1}^n w_i*{\#event}_i*d\left({\#event}_i\right) = \left(w_1*{\#event}_1*d\left({\#event}_1\right)\right)+\left(w_2*{\#event}_2*d\left({\#event}_2\right)\right)+\dots+\left(w_n*{\#event}_n*d\left({\#event}_n\right)\right)$$

### Computing decay

In [39]:
import datetime
from datetime import date, timedelta, datetime

def compute_decay(eventDate, decayDays):
    age = (date.today() - datetime.strptime(eventDate, '%d/%m/%Y %H:%M').date()) // timedelta(days=decayDays)
    #print("Age of event:", age)
    decay = 1/age #simple decay
    #print("Decay factor:", decay)

    return decay

createdEvent = evidence.at[0,'created']
thresholdDays = 2 # Number of days
decayFactor = compute_decay(createdEvent, thresholdDays)

print(decayFactor)

0.0010834236186348862


In [40]:
newMatrix1 = pd.DataFrame(columns=content, index=users)

for user in users:
  for item in content:
    userEvidence = evidence['user_id'] == user
    contentEvidence = evidence['content_id'] == item
    userItemEvents = evidence[userEvidence & contentEvidence]
    updatedValue = 0
    for _, row in userItemEvents.iterrows():
        w = eventWeights[row['event']]
        decay = compute_decay(row['created'], thresholdDays)
        updatedValue += w * decay
    newMatrix1.at[user, item] = updatedValue

print(newMatrix1)

         4501244   3521164   3640424   2823054   3553976    3470600   4513674  \
400003  5.222102  5.032503  4.452871  4.198267  3.017335   4.512459  7.659805   
400005         0         0  8.185265         0  8.434453          0  8.494041   
400001  5.211268  4.972914  4.609967  4.333694  2.529794   5.666306  7.394366   
400006  8.250271  8.315276  0.211268  9.425785         0  10.010834  9.572048   
400002  2.432286  2.621885  6.381365  3.049837  1.852654   2.540628  4.349946   
400004         0         0  9.014085         0         0          0         0   

         4698684   3315342    3874544  ...   4196776   2948356   1355644  \
400003  4.501625  4.247021   4.550379  ...  1.760563  2.210184  2.973998   
400005         0  8.629469          0  ...  0.054171  0.086674  8.266522   
400001  4.897075  4.588299   4.815818  ...   1.76598  1.858072  2.329361   
400006  9.317443  0.032503  10.335861  ...  0.065005  0.249187         0   
400002  2.058505  6.673889   2.340195  ...  5.200433

- Course Instructor: Dr. Paolo Mengoni (Senior Lecturer at Department of Interactive Media, Hong Kong Baptist University)
  - pmengoni@hkbu.edu.hk

- The codes in this notebook take insipiration from various sources. All codes are for educational purposes only and released under the CC1.0.